In [1]:
from pathlib import Path

import numpy as np

In [2]:
input_data = Path("example.txt").read_text()
lines = input_data.splitlines()
coordinates = np.array([line.split(",") for line in lines], dtype=int)
print(coordinates[:10])

[[ 7  1]
 [11  1]
 [11  7]
 [ 9  7]
 [ 9  5]
 [ 2  5]
 [ 2  3]
 [ 7  3]]


## Part I

In [3]:
observations, dimensions = coordinates.shape
distances = np.zeros((observations, observations), dtype=int)


def compute_distance(x: np.ndarray, y: np.ndarray) -> np.ndarray:
    return (np.abs(x - y) + 1).prod(axis=1)


for i in range(observations):
    distances[i] = compute_distance(coordinates, coordinates[i])

distances = np.triu(distances, k=1)

print(distances)

[[ 0  5 35 21 15 30 18  3]
 [ 0  0  7 21 15 50 30 15]
 [ 0  0  0  3  9 30 50 25]
 [ 0  0  0  0  3 24 40 15]
 [ 0  0  0  0  0  8 24  9]
 [ 0  0  0  0  0  0  3 18]
 [ 0  0  0  0  0  0  0  6]
 [ 0  0  0  0  0  0  0  0]]


In [4]:
print(distances.max())

50


## Part II

In [5]:
from itertools import pairwise

from skimage.morphology import flood_fill


In [6]:
sorted_indices = np.dstack(
    np.unravel_index(
        np.argsort(np.ravel(distances), axis=0),
        distances.shape,
    ),
)[0][::-1]
print(sorted_indices[:5])

[[1 5]
 [2 6]
 [3 6]
 [0 2]
 [1 6]]


In [7]:
offset = coordinates[:, 0].min(), coordinates[:, 1].min()
maximum = coordinates[:, 0].max(), coordinates[:, 1].max()
shape = (maximum[0] - offset[0] + 1, maximum[1] - offset[1] + 1)

offset, maximum, shape

((np.int64(2), np.int64(1)),
 (np.int64(11), np.int64(7)),
 (np.int64(10), np.int64(7)))

In [8]:
unique_coords_0 = np.sort(np.unique(coordinates[:, 0])) - offset[0]
print("0:", unique_coords_0)
delta_0 = np.roll(unique_coords_0, -1) - unique_coords_0
delta_0[-1] = 0
print("0 delta:", delta_0)

unique_coords_1 = np.sort(np.unique(coordinates[:, 1])) - offset[1]
print("1:", unique_coords_1)
delta_1 = np.roll(unique_coords_1, -1) - unique_coords_1
delta_1[-1] = 0
print("1 delta:", delta_1)

compressed = np.ones((unique_coords_0.size, unique_coords_1.size), dtype=int)
for i, d0 in enumerate(delta_0):
    if d0 > 1:
        compressed[i + 1] += d0
for j, d1 in enumerate(delta_1):
    if d1 > 1:
        compressed[:, j + 1] += d1
print(compressed)

0: [0 5 7 9]
0 delta: [5 2 2 0]
1: [0 2 4 6]
1 delta: [2 2 2 0]
[[1 3 3 3]
 [6 8 8 8]
 [3 5 5 5]
 [3 5 5 5]]


In [9]:
def compress_coordinate(coord: tuple[int, int]) -> tuple[int, int]:
    x, y = coord
    x_index = np.searchsorted(unique_coords_0, x - offset[0])
    y_index = np.searchsorted(unique_coords_1, y - offset[1])
    return x_index, y_index


compressed_coordinates = np.array(
    [compress_coordinate((x, y)) for x, y in coordinates],
    dtype=int,
)
print(compressed_coordinates)

[[1 0]
 [3 0]
 [3 3]
 [2 3]
 [2 2]
 [0 2]
 [0 1]
 [1 1]]


In [10]:
compressed_mask = np.ones(compressed.shape, dtype=int)
for x, y in compressed_coordinates:
    compressed_mask[x, y] = 2


connections = [
    (compressed_coordinates[-1], compressed_coordinates[0]),
    *pairwise(compressed_coordinates),
]

# connect the points in order
for coordinate_1, coordinate_2 in connections:
    x1, y1 = coordinate_1
    x2, y2 = coordinate_2
    if x1 == x2:
        for y in range(min(y1, y2), max(y1, y2) + 1):
            compressed_mask[x1, y] = 2
    elif y1 == y2:
        for x in range(min(x1, x2), max(x1, x2) + 1):
            compressed_mask[x, y1] = 2
    else:
        raise ValueError("Only horizontal or vertical lines are supported")


compressed_mask = np.pad(
    compressed_mask,
    pad_width=1,
    mode="constant",
    constant_values=1,
)

compressed_mask = flood_fill(compressed_mask, (0, 0), 0)
compressed_mask = compressed_mask[1:-1, 1:-1]
compressed_mask = compressed_mask > 0

print(compressed_mask.astype(int))

[[0 1 1 0]
 [1 1 1 0]
 [1 1 1 1]
 [1 1 1 1]]


In [11]:
found = False
for index_1, index_2 in sorted_indices:
    coordinate_1 = coordinates[index_1]
    coordinate_2 = coordinates[index_2]

    compressed_coord_1 = compress_coordinate(tuple(coordinate_1))
    compressed_coord_2 = compress_coordinate(tuple(coordinate_2))

    upper_left = (
        min(compressed_coord_1[0], compressed_coord_2[0]),
        min(compressed_coord_1[1], compressed_coord_2[1]),
    )
    lower_right = (
        max(compressed_coord_1[0], compressed_coord_2[0]) + 1,
        max(compressed_coord_1[1], compressed_coord_2[1]) + 1,
    )
    region = compressed_mask[
        upper_left[0] : lower_right[0],
        upper_left[1] : lower_right[1],
    ]
    if np.all(region):
        found = True
        break

print("Found valid pair:" if found else "No valid pair found")
print(coordinate_1, coordinate_2)
area = compute_distance(
    coordinate_1.reshape(1, -1),
    coordinate_2.reshape(1, -1),
)[0]
print(area)

Found valid pair:
[9 5] [2 3]
24
